In [ ]:
from random import choice
from itertools import chain
import matplotlib.pyplot as plt
import numpy as np
from graphviz import Graph

%matplotlib inline

In [ ]:
N_LEN = 2 ** 3  # how long is a number?
DIALECT = ("AB", "CD")
ALPHABET = tuple(chain.from_iterable([[l + r for r in DIALECT[1]] for l in DIALECT[0]]))
DELTA = 1
ALPHABET

In [ ]:
def graph(network, render=None):
    dot = Graph(engine="neato")
    dot.format = 'png'
    for edge in network:
        dot.edge(*edge)
    if render is not None:
        dot.render(render)
    return dot

In [ ]:
def number(seq, dialect=("AB", "CD"), delta=1):
    l = dialect[0]
    plus = set(l[:len(l) // 2])
    n = 0
    for l, _ in seq:
        delta /= 2
        n = n + (delta if l in plus else -delta)
    return n

In [ ]:
alphabet_iter = iter(ALPHABET)
NETWORK = next(alphabet_iter)
GROUP = next(alphabet_iter)
LAYER = next(alphabet_iter)

alphabet_iter = iter(ALPHABET)
TYPEA = next(alphabet_iter)
TYPEB = next(alphabet_iter)
TYPEC = next(alphabet_iter)
TYPED = next(alphabet_iter)
types = {TYPEA, TYPEB, TYPEC, TYPED}

n_id = 0
g_id = 0
l_id = 0


def read_count(seq):
    count = None
    if len(seq) >= 8:
        count = int(round(number(seq[:8], delta=2) + 3))
        seq = seq[8:]
    return seq, count


def process_group(seq, p, connections):
    global g_id
    seq, count = read_count(seq)
    if count is not None:
        node_id = "G{}".format(g_id)
        g_id += 1
        # print("group", node_id, count)
        connections.append((p, node_id))
        for _ in range(count):
            seq = process_layer(seq, node_id, connections)
    return seq

def process_layer(seq, p, connections):
    global l_id
    node_id = str(l_id)
    l_id += 1
    layer_type = None
    while layer_type is None and seq:
        element = seq[0]
        seq = seq[1:]
        if element in types:
            layer_type = element
    # print("layer", node_id, 'type', layer_type)
    connections.append((p, 'L ' + layer_type + node_id))
    return seq


def process_network(seq, p, connections):
    global n_id
    if seq:
        node_id = "N{}".format(n_id)
        n_id += 1
        # print("network", node_id)
        command = seq[0]
        seq = seq[1:]
        seq, count = read_count(seq)
        if count is not None:
            connections.append((p, node_id))
            if command == NETWORK:
                for _ in range(count):
                    seq = process_network(seq, node_id, connections)
            if command == GROUP:
                for _ in range(count):
                    seq = process_group(seq, node_id, connections)
    return seq

def process_sequence(seq, connections):
    node_id = "R"
    last_len = len(seq)
    while seq:
        seq = process_network(seq, node_id, connections)
        if len(seq) == last_len:
            seq = None  # nothing can be done anymore


In [ ]:
network = []
test_seq = [choice(ALPHABET) for _ in range(100)]
process_sequence(test_seq, network)
graph(network)
# connections



In [ ]:
from IPython.display import display
network1 = []
network2 = []
network3 = []
test_seq1 = [choice(ALPHABET) for _ in range(200)]
process_sequence(test_seq1, network1)
test_seq2 = [choice(ALPHABET) for _ in range(200)]
process_sequence(test_seq2, network2)

test_seq3 = [l1 + r2 for (l1, _) , (_, r2) in zip(test_seq1, test_seq2)]
process_sequence(test_seq3, network3)

display(graph(network1, 'p1'))
display(graph(network2, 'p2'))
display(graph(network3, 'c'))
# connections

